In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("temp_measurements.csv")

In [3]:
df.head()

,date,city,temperature
0,2023-08-20 13:38:00,Barcelona,10
1,2023-06-20 13:55:00,Ljubljana,23
2,2022-03-26 13:24:00,Ljubljana,5
3,2023-11-28 15:49:00,Rome,-4
4,2023-10-2 13:12:00,Ljubljana,7


In [4]:
df.describe()

,temperature
count,858.000000
mean,13.257576
std,11.047583
min,-10.000000
25%,5.000000
50%,13.000000
75%,21.000000
max,40.000000


In [6]:
df.dtypes

date           object
city           object
temperature     int64
dtype: object

In [7]:
# Transform the date column to a datetime object 
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")

### What if we had a dataset with dirty data? 
#### How would we make sure that our code didn't break, throw an exception/error, or return incorrect results?
Example:

In [20]:
df_dirty = pd.read_csv("temp_measurements_dirty_data.csv")

In [21]:
df_dirty.head()

,date,city,temperature
0,2023-08-20 13:38:00,Barcelona,10
1,2023-06-20-13:55:00,Ljubljana,23
2,2022-03-26-13:24:00,Ljubljana,5
3,2023-11-28 15:49:00,Rome,-4
4,2023-10-2 13:12:00,Ljubljana,7


In [17]:
df_dirty["date"] = pd.to_datetime(df_dirty["date"], format="%Y-%m-%d %H:%M:%S")

ValueError: time data "2023-06-20-13:55:00" doesn't match format "%Y-%m-%d %H:%M:%S", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [18]:
df_dirty["date"] = pd.to_datetime(df_dirty["date"])

ValueError: time data "2023-06-20-13:55:00" doesn't match format "%Y-%m-%d %H:%M:%S", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [19]:
print(pd.to_datetime(df_dirty["date"], errors="coerce"))

0     2023-08-20 13:38:00
1                     NaT
2                     NaT
3     2023-11-28 15:49:00
4     2023-10-02 13:12:00
              ...        
853   2022-03-25 11:13:00
854   2022-11-17 13:06:00
855   2023-02-22 15:48:00
856   2022-12-05 14:16:00
857   2023-02-08 10:27:00
Name: date, Length: 858, dtype: datetime64[ns]


In [22]:
print(df_dirty["temperature"].astype(int, errors="ignore"))

0      10
1      23
2       5
3      -4
4       7
       ..
853    12
854     9
855    -2
856    -7
857    13
Name: temperature, Length: 858, dtype: object


In [24]:
print(df_dirty["temperature"].astype(int, errors="raise"))

ValueError: invalid literal for int() with base 10: '22a'

### Back to initial clean dataset
Suppose we had to convert the temperature from Celsius to Fahrenheit to localize the data for a US-based client.

How would you go about creating a new column to convert the temperature from Celsius to Fahrenheit?



#### Celsius to Fahrenheit formula: 
##### (°C × 9/5) + 32 = °F

In [25]:
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32

df["temperature_fahrenheit"] = df["temperature"].apply(celsius_to_fahrenheit)

In [26]:
df.head()

,date,city,temperature,temperature_fahrenheit
0,2023-08-20 13:38:00,Barcelona,10,50.0
1,2023-06-20 13:55:00,Ljubljana,23,73.4
2,2022-03-26 13:24:00,Ljubljana,5,41.0
3,2023-11-28 15:49:00,Rome,-4,24.8
4,2023-10-02 13:12:00,Ljubljana,7,44.6


### Focusing on the data for one particular city 
Let's say we want to focus only on the data for Barcelona. 

Ex. 1: Manual maneuver of the data

In [27]:
df.count()

date                      858
city                      858
temperature               858
temperature_fahrenheit    858
dtype: int64

In [28]:
df["city"].value_counts()

city
Barcelona    182
Ljubljana    179
Vienna       167
Paris        167
Rome         163
Name: count, dtype: int64

In [32]:
df_bcn = pd.DataFrame(df[df["city"] == "Barcelona"])

In [33]:
df_bcn.count()

date                      182
city                      182
temperature               182
temperature_fahrenheit    182
dtype: int64

In [34]:
# Extract month from date 
df_bcn["month"] = df_bcn["date"].dt.month

In [37]:
## Show the average, max and min temperature for July

print(df_bcn[df_bcn["month"] == 7]["temperature"].mean())


24.444444444444443


In [38]:
print(df_bcn[df_bcn["month"] == 7]["temperature"].max())

38


In [39]:
print(df_bcn[df_bcn["month"] == 7]["temperature"].min())

10


Ex. 2: By using groupby method to first get Barcelona data and then calculate the statistics

In [40]:
df_bcn_grouped = df.groupby("city").get_group("Barcelona")

In [41]:
df_bcn_grouped.head()

,date,city,temperature,temperature_fahrenheit
0,2023-08-20 13:38:00,Barcelona,10,50.0
6,2022-10-05 09:49:00,Barcelona,0,32.0
7,2023-03-22 12:57:00,Barcelona,-4,24.8
8,2022-10-09 10:19:00,Barcelona,20,68.0
9,2022-02-25 13:05:00,Barcelona,9,48.2


In [42]:
df_bcn_monthly_stats = df_bcn.groupby("month")["temperature"].agg(["mean", "max", "min"])

In [43]:
df_bcn_monthly_stats

,mean,max,min
month,,,
1,4.555556,13,-5
2,7.687500,20,-5
3,6.833333,19,-4
4,14.705882,25,2
5,18.647059,27,7
6,22.291667,35,11
7,24.444444,38,10
8,24.800000,40,10
9,17.266667,30,7
